# Evaluation of the Recommender Model  
In this notebook, I evaluated the recommender and the recommendations it output.  I also added a biased output of the recommender to see if there were some healthier options that were similar to the recipes the user likes and input into the recommender.

In [4]:
#Libraries for general analysis and data set manipulation:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [15]:
#df_test = pd.read_csv('./data/recommender_final.csv', nrows=100, index_col = 'url')
recommender_test = pd.read_csv('./data/recommender_final.csv', nrows=100, index_col = 'url')

In [16]:
#Checking the values in the recommender are floats and collecting the column names for the engine
#to speed up reading of the csv:
recommender_test.dtypes.value_counts()

float64    22941
dtype: int64

In [61]:
#Inspecting the head of the beginning of the recommender dataframe:
recommender_test.head()

,http://www.seriouseats.com/recipes/2010/06/the-martini-recipe.html,http://www.marthastewart.com/337857/pasta-dough,http://www.saveur.com/article/Recipes/Negroni-Cocktail,http://food52.com/recipes/27825-simple-fresh-pasta,http://www.epicurious.com/recipes/food/views/Egg-Noodle-351712,http://www.seriouseats.com/recipes/2011/02/time-for-a-drink-pisco-sour.html,http://www.marthastewart.com/354987/fresh-pasta,https://food52.com/recipes/10620-cauliflower-popcorn,http://www.saveur.com/article/Recipes/Manhattan-1000088919,http://www.seriouseats.com/recipes/2011/11/how-to-make-mulled-cider.html,...,http://www.recipezaar.com/Old-fashioned-Linguine-with-White-Clam-Sauce-17722,http://www.food.com/recipe/seafood-casserole-for-2-252748,http://www.myrecipes.com/recipe/chicken-dumplings-13,http://www.delish.com/cooking/recipe-ideas/recipes/a31279/greek-chicken-pitas-recipe-ghk0411/,https://www.foodnetwork.com/recipes/anne-burrell/linguine-with-white-clam-sauce-recipe-1919333,http://www.delish.com/cooking/recipe-ideas/recipes/a30834/quick-mu-shu-pork-recipe-ghk0910/,http://www.food.com/recipe/red-lobster-tartar-sauce-335329,http://www.foodista.com/recipe/F3RYN6B2/healthy-hazelnut-cookies,https://www.allrecipes.com/recipe/22188/yeast-doughnuts/,http://www.delish.com/cooking/recipe-ideas/recipes/a26274/spicy-southern-fried-chicken-3300/
url,,,,,,,,,,,,,,,,,,,,,
http://www.seriouseats.com/recipes/2010/06/the-martini-recipe.html,1.0000,0.1960,0.8100,0.3162,0.2850,0.7770,0.3596,0.2554,0.7793,0.48140,...,-0.07715,-0.11890,-0.1295,-0.09186,0.09580,-0.007960,0.14550,-0.1494,-0.081200,-0.08203
http://www.marthastewart.com/337857/pasta-dough,0.1960,1.0000,0.1476,0.4390,0.4192,0.1220,0.7200,0.1396,0.1364,0.06366,...,-0.02808,-0.08640,-0.0903,-0.10840,0.05603,-0.062380,0.05084,-0.1467,0.039300,-0.03029
http://www.saveur.com/article/Recipes/Negroni-Cocktail,0.8100,0.1476,1.0000,0.2670,0.2385,0.6190,0.3086,0.2502,0.9634,0.23960,...,-0.10046,-0.11910,-0.1038,-0.09260,0.06140,-0.008736,0.14100,-0.1401,-0.068700,-0.07040
http://food52.com/recipes/27825-simple-fresh-pasta,0.3162,0.4390,0.2670,1.0000,0.6600,0.2377,0.7040,0.4130,0.2517,0.22350,...,-0.06730,-0.13000,-0.1147,-0.11600,0.04123,-0.071800,0.14930,-0.2412,-0.001492,-0.06506
http://www.epicurious.com/recipes/food/views/Egg-Noodle-351712,0.2850,0.4192,0.2385,0.6600,1.0000,0.2130,0.6670,0.1700,0.2246,0.19130,...,-0.05830,-0.12195,-0.1105,-0.11250,0.03415,-0.063300,0.11880,-0.1927,0.010704,-0.05734


In [19]:
#Saving the column names to float_cols and checking there are 22941 column names (number of recipes have):
float_cols = recommender_test.columns
len(float_cols)

22941

In [18]:
#Reading in the recommender (cosine similarity matrix) to a dataframe called recommender:
#Using an engine to take in float values as float16 values instead of float64 values to increase
#speed of reading in the recommender dataframe:
float16_cols = {c: np.float16 for c in float_cols}

recommender = pd.read_csv('./data/recommender_final.csv', index_col = 'url', engine='c', dtype=float16_cols)

#Sources:
#https://chrisalbon.com/python/data_wrangling/pandas_dataframe_importing_csv/
#https://stackoverflow.com/questions/30494569/how-to-force-pandas-read-csv-to-use-float32-for-all-float-columns

In [22]:
#Importing recipes dataframe with the added nutition score feature which will be used to bias results to be
#more healthy:
df_recs = pd.read_csv('./data/df_recs_clean_final_nutr_sc.csv')

In [134]:
#Increasing the number of columns that will be displayed when inspecting the head of a dataframe:
pd.set_option('display.max_columns', 350)
#Source: https://stackoverflow.com/questions/11707586/how-do-i-expand-the-output-display-to-see-more-columns

In [135]:
#Checking df_recs looks alright:
df_recs.head()

,title,url,yield,total_weight,total_time,calories_per_serv,total_weight_per_serv,total_time_per_serv,monounsat_fat_g_per_serv,polyunsat_fat_g_per_serv,trans_fat_g_per_serv,sugar_g_per_serv,calcium_pct_div100_per_serv,carbs_pct_div100_per_serv,cholesterol_pct_div100_per_serv,energy_pct_div100_per_serv,sat_fat_pct_div100_per_serv,fat_pct_div100_per_serv,iron_pct_div100_per_serv,fiber_pct_div100_per_serv,folate_pct_div100_per_serv,potassium_pct_div100_per_serv,magnesium_pct_div100_per_serv,sodium_pct_div100_per_serv,niacin_pct_div100_per_serv,phosphorus_pct_div100_per_serv,protein_pct_div100_per_serv,riboflavin_pct_div100_per_serv,thiamin_pct_div100_per_serv,vit_A_pct_div100_per_serv,vit_B6_pct_div100_per_serv,vit_B12_pct_div100_per_serv,vit_C_pct_div100_per_serv,vit_D_pct_div100_per_serv,vit_E_pct_div100_per_serv,vit_K_pct_div100_per_serv,source_BigOven,source_Cookstr,source_Delish,source_Epicurious,source_Food Network,source_Food52,source_Foodista,source_Good Housekeeping,source_Group Recipes,source_Kitchen Daily,source_Kraft Foods,source_Martha Stewart,source_My Recipes,source_Saveur,source_Serious Eats,source_Taste of Home,source_Williams-Sonoma,source_food.com,source_recipezaar.com,source_nan,diet_labels_Balanced,diet_labels_High-Fiber,diet_labels_High-Protein,diet_labels_Low-Carb,diet_labels_Low-Fat,diet_labels_Low-Sodium,health_labels_Alcohol-Cocktail,health_labels_Alcohol-Free,health_labels_Celery-Free,health_labels_Crustacean-Free,health_labels_Dairy-Free,health_labels_Egg-Free,health_labels_Fish-Free,health_labels_Gluten-Free,health_labels_Keto-Friendly,health_labels_Kidney-Friendly,health_labels_Kosher,health_labels_Low Potassium,health_labels_Low Sugar,health_labels_Lupine-Free,health_labels_Mollusk-Free,health_labels_Mustard-Free,health_labels_No oil added,health_labels_Paleo,health_labels_Peanut-Free,health_labels_Pescatarian,health_labels_Pork-Free,health_labels_Red-Meat-Free,health_labels_Sesame-Free,health_labels_Shellfish-Free,health_labels_Soy-Free,health_labels_Sugar-Conscious,health_labels_Tree-Nut-Free,health_labels_Vegan,health_labels_Vegetarian,health_labels_Wheat-Free,cautions_FODMAP,cautions_Sulfites,cuisine_type_.,cuisine_type_4 Points,cuisine_type_African,cuisine_type_Albanian,cuisine_type_Amerian,cuisine_type_American,"cuisine_type_American, Barbecue, Southern","cuisine_type_American, French","cuisine_type_American, Italian","cuisine_type_American, Southern",cuisine_type_American-South,cuisine_type_American|Asian,cuisine_type_American|North American|Cajun/Creole,cuisine_type_Amish,cuisine_type_Andhra,cuisine_type_Appetizer,cuisine_type_Arab,cuisine_type_Argentinian,cuisine_type_Armenian,cuisine_type_Asian,"cuisine_type_Asian, Chinese",cuisine_type_Aussie,cuisine_type_Australian,cuisine_type_Australian/New Zealand,cuisine_type_Austrian,cuisine_type_Baking,cuisine_type_Balochi,cuisine_type_Bangladeshi,cuisine_type_Belarusian,cuisine_type_Belgian,cuisine_type_Belgium,cuisine_type_Brazilian,cuisine_type_Breakfast,cuisine_type_British,cuisine_type_BritishEurope,cuisine_type_Bulgarian,cuisine_type_Cajun,cuisine_type_Cajun/Creole|Southern,cuisine_type_Canadian,cuisine_type_Caribbean,cuisine_type_Central American/Caribbean,cuisine_type_Cheap,cuisine_type_Chechen,cuisine_type_Chinese,cuisine_type_Colombian,cuisine_type_Cookie,cuisine_type_Costa Rican,cuisine_type_Creole,cuisine_type_Cuban,cuisine_type_Cuban|Central American/Caribbean,cuisine_type_Curry,cuisine_type_Danish,cuisine_type_Dessert,cuisine_type_Dominican,cuisine_type_Dutch,cuisine_type_Earth,cuisine_type_Eastern Europe,cuisine_type_Eastern European,cuisine_type_Ecuadorian,cuisine_type_Egypt,cuisine_type_English,cuisine_type_Estern,cuisine_type_Europe,cuisine_type_European,"cuisine_type_European, French, Paris","cuisine_type_European, Italian","cuisine_type_European, Scottish",cuisine_type_Filipino,cuisine_type_France,cuisine_type_French,cuisine_type_GAPS,cuisine_type_German,cuisine_type_Goan,cuisine_type_Gourment,cuisine_type_Greek,cuisine_type_Hawa

In [26]:
#I feel like eating something from Belgium.  What Belgian foods are in df_recs?
df_recs[df_recs['cuisine_type_Belgian'] == 1]['url'].head(10)

2067    http://www.marthastewart.com/343477/yogurt-waf...
2197    http://www.saveur.com/article/Recipes/Belgian-...
2206    http://www.epicurious.com/recipes/food/views/o...
2227       https://food52.com/recipes/8231-raised-waffles
2291     http://www.cookstr.com/recipes/sourdough-waffles
3032    http://www.kitchendaily.com/recipe/brownie-waf...
3073    http://allrecipes.com/Recipe/Brown-Sugar-Bacon...
3161    http://www.foodista.com/recipe/6SWXKT4W/raised...
3162     http://www.recipezaar.com/Belgian-Waffles-227109
3212    http://www.goodhousekeeping.com/food-recipes/a...
Name: url, dtype: object

In [28]:
#Let's take a look at the recipe at index 2197:
df_recs.iloc[2197]['url']

'http://www.saveur.com/article/Recipes/Belgian-Waffles'

This recipe looks delicious!  I'd like to find similar recipes and to see what happens when I sort by health score (bias toward a healthier option).

In [62]:
#Looking at recipes most similar to a recipe I am interested in:
recommender['http://www.saveur.com/article/Recipes/Belgian-Waffles'].sort_values(ascending=False).head(10)

url
http://www.saveur.com/article/Recipes/Belgian-Waffles                            1.000000
http://www.saveur.com/article/Recipes/Buttermilk-Waffles                         0.961426
https://www.allrecipes.com/recipe/17681/belgian-waffles/                         0.942383
http://allrecipes.com/Recipe/Waffle-Cookies-I/Detail.aspx                        0.939453
http://www.food.com/recipe/classic-belgian-waffle-490894                         0.937988
http://www.food.com/recipe/the-best-belgian-waffles-63071                        0.927734
https://food52.com/recipes/8231-raised-waffles                                   0.925781
https://www.foodnetwork.com/recipes/waffle-of-insane-greatness-recipe-1943589    0.924316
http://www.recipezaar.com/Overnight-Waffles-16235                                0.923828
http://www.foodista.com/recipe/6SWXKT4W/raised-waffles                           0.920410
Name: http://www.saveur.com/article/Recipes/Belgian-Waffles, dtype: float16

In [108]:
#Creating a biased health recommender:
def recommender_health_bias(url):
    print(f"Health score of original recipe: {df_recs[df_recs['url'] == url]['nutrition_score']}")
          
    print(f'Top 10 results before adding the health bias: {recommender[url].sort_values(ascending=False).head(10)}')
    top_100_info = recommender[url].sort_values(ascending=False).head(100)
    top_100 = recommender[url].sort_values(ascending=False).head(100).index
    
    df_top_100_nutr = pd.DataFrame(columns=df_recs.columns)
    for i, row in df_recs.iterrows():
        if row['url'] in top_100:
            df_top_100_nutr.loc[i] = row
    
    
    top_10_nutr_bias = list(df_top_100_nutr.sort_values(by = 'nutrition_score', ascending = False)['url'].head(10))
          
    print(f'Top 10 results after adding the health bias: {top_100_info[top_10_nutr_bias]}') 
    print('Full information about the top 10 results after adding the health bias:')
    return df_top_100_nutr.sort_values(by = 'nutrition_score', ascending = False).head(10)
              

In [109]:
url = 'http://www.saveur.com/article/Recipes/Belgian-Waffles'
recommender_health_bias(url)

Health score of original recipe: 2197    7
Name: nutrition_score, dtype: int64
Top 10 results before adding the health bias: url
http://www.saveur.com/article/Recipes/Belgian-Waffles                            1.000000
http://www.saveur.com/article/Recipes/Buttermilk-Waffles                         0.961426
https://www.allrecipes.com/recipe/17681/belgian-waffles/                         0.942383
http://allrecipes.com/Recipe/Waffle-Cookies-I/Detail.aspx                        0.939453
http://www.food.com/recipe/classic-belgian-waffle-490894                         0.937988
http://www.food.com/recipe/the-best-belgian-waffles-63071                        0.927734
https://food52.com/recipes/8231-raised-waffles                                   0.925781
https://www.foodnetwork.com/recipes/waffle-of-insane-greatness-recipe-1943589    0.924316
http://www.recipezaar.com/Overnight-Waffles-16235                                0.923828
http://www.foodista.com/recipe/6SWXKT4W/raised-waffles       

,title,url,yield,total_weight,total_time,calories_per_serv,total_weight_per_serv,total_time_per_serv,monounsat_fat_g_per_serv,polyunsat_fat_g_per_serv,...,ingredient_category_water,ingredient_category_wines,ingredient_category_yogurt,ingredient_category_condiments_and_sauces,ingredient_category_cured_meats,ingredient_category_frozen_poultry,ingredient_category_plant_based_protein,ingredient_category_liquors_and_cocktails,ingredient_category_sugar_and_syrups,nutrition_score
13324,Old-Fashioned Buttermilk Waffles,http://www.williams-sonoma.com/recipe/old-fash...,4.0,991.839806,0.0,396.653532,247.959951,0.000000,3.748666,1.911706,...,0,0,0,2,0,0,0,0,0,14
3212,Belgian Waffles,http://www.goodhousekeeping.com/food-recipes/a...,6.0,1003.635462,0.0,299.920667,167.272577,0.000000,2.911004,1.111719,...,0,0,0,2,0,0,0,0,0,13
9251,Overnight Waffles,http://www.recipezaar.com/Overnight-Waffles-16235,4.0,968.190720,0.0,578.817991,242.047680,0.000000,10.192534,2.112940,...,0,0,0,2,0,0,0,0,0,13
2170,Breakfast Bread,http://www.saveur.com/article/Recipes/Breakfas...,12.0,1477.525000,0.0,260.940896,123.127083,0.000000,1.147282,0.750674,...,0,0,0,2,0,0,0,0,0,13
2291,Sourdough Waffles,http://www.cookstr.com/recipes/sourdough-waffles,8.0,1014.150000,0.0,296.615437,126.768750,0.000000,1.011549,0.963111,...,0,0,0,2,0,0,0,0,0,13
13388,Buttermilk Waffles Recipe,http://www.grouprecipes.com/126604/buttermilk-...,4.0,992.500000,53.0,541.252750,248.125000,13.250000,7.244734,1.639751,...,0,0,0,2,0,0,0,0,0,13
9461,Belgian Waffles,https://www.allrecipes.com/recipe/17681/belgia...,8.0,1645.872066,95.0,511.588811,205.734008,11.875000,5.901935,1.296633,...,0,0,0,2,0,0,0,0,0,12
4144,Waffle Or Pancake Mix,http://www.foodista.com/recipe/2437QZKM/waffle...,1.0,372.183150,0.0,584.606666,372.183150,0.000000,23.714705,6.098778,...,0,0,0,2,0,0,0,0,0,12
2067,Yogurt Waffles,http://www.marthastewart.com/343477/yogurt-waf...,6.0,822.650000,72.0,276.468167,137.108333,12.000000,2.841810,1.055318,...,0,0,1,2,0,0,0,0,0,12
9695,Basic Batter Waffles,http://www.food.com/recipe/basic-batter-waffle...,6.0,874.821245,20.0,395.415222,145.803541,3.333333,11.154013,2.745272,...,0,0,0,2,0,0,0,0,0,12


The unbiased recommendations look great!  There are a lot of types of waffles that are similar to the initial waffle recipe input into the recommender.  The healthier options look good too, and it's great that the health score doubled from 7 to 14 when the recommender was biased toward healthier recommended outputs.

In [110]:
#I feel like eating some Mexican food.  What Mexican foods are in df_recs?
df_recs[df_recs['cuisine_type_Mexican'] == 1]['url'].head(10)

32     http://www.seriouseats.com/recipes/2014/11/15-...
78     http://www.marthastewart.com/336643/nacho-pota...
81     http://www.marthastewart.com/333225/spicy-pull...
95     http://www.epicurious.com/recipes/food/views/P...
104    http://www.seriouseats.com/recipes/2014/12/the...
168       https://food52.com/recipes/12850-avocado-cream
179    http://www.marthastewart.com/344471/beef-enchi...
237    http://food52.com/recipes/32638-chicken-tamale...
246         https://food52.com/recipes/997-carne-guisada
359    http://www.kitchendaily.com/recipe/turkey-ench...
Name: url, dtype: object

In [111]:
#Let's take a look at the recipe at index 32:
df_recs.iloc[32]['url']

'http://www.seriouseats.com/recipes/2014/11/15-minute-turkey-mole-enchilada-recipe.html'

This recipe looks delicious!  I'd like to find similar recipes and to see what happens when I sort by health score (bias toward a healthier option).

In [112]:
url = df_recs.iloc[32]['url']
recommender_health_bias(url)

Health score of original recipe: 32    8
Name: nutrition_score, dtype: int64
Top 10 results before adding the health bias: url
http://www.seriouseats.com/recipes/2014/11/15-minute-turkey-mole-enchilada-recipe.html              1.000000
http://www.seriouseats.com/recipes/2012/01/how-to-make-cupcakes-in-an-ice-cream-cone-recipe.html    0.693848
http://www.seriouseats.com/recipes/2010/01/pulled-pork-nachos-super-bowl-recipe.html                0.689941
http://www.seriouseats.com/recipes/2012/05/gluten-free-flour-tortillas.html                         0.665527
http://www.seriouseats.com/recipes/2010/06/jim-laheys-no-knead-pizza-dough-recipe.html              0.617188
http://www.seriouseats.com/recipes/2012/03/chess-pie-recipe.html                                    0.614746
http://www.seriouseats.com/recipes/2014/12/the-best-chicken-enchiladas-recipe.html                  0.608398
http://www.seriouseats.com/recipes/2012/03/sweet-potato-pie-recipe.html                             0.608398
h

,title,url,yield,total_weight,total_time,calories_per_serv,total_weight_per_serv,total_time_per_serv,monounsat_fat_g_per_serv,polyunsat_fat_g_per_serv,...,ingredient_category_water,ingredient_category_wines,ingredient_category_yogurt,ingredient_category_condiments_and_sauces,ingredient_category_cured_meats,ingredient_category_frozen_poultry,ingredient_category_plant_based_protein,ingredient_category_liquors_and_cocktails,ingredient_category_sugar_and_syrups,nutrition_score
6284,Enchiladas Suizas Recipe,http://www.seriouseats.com/recipes/2010/12/enc...,4.0,2622.142370,60.0,597.716390,655.535593,15.000,12.471902,5.777512,...,0,0,0,0,0,0,0,0,0,19
104,The Best Creamy Chicken Enchiladas Recipe,http://www.seriouseats.com/recipes/2014/12/the...,6.0,3251.112044,90.0,784.292919,541.852007,15.000,18.279583,7.837750,...,0,0,0,1,0,0,0,0,0,15
27,No-Knead Pizza Dough Recipe,http://www.seriouseats.com/recipes/2010/06/jim...,4.0,812.485020,0.0,480.304750,203.121255,0.000,9.973214,1.987738,...,1,0,0,2,0,0,0,0,0,13
52,Black Eyed Peas with Ham Hock Recipe,http://www.seriouseats.com/recipes/2011/04/bla...,8.0,1506.229018,180.0,248.587905,188.278627,22.500,6.315130,1.910263,...,0,0,0,1,1,0,0,0,0,13
6335,The Best White Chili With Chicken Recipe,http://www.seriouseats.com/recipes/2014/10/whi...,8.0,3215.789137,1440.0,454.663358,401.973642,180.000,9.548972,2.754246,...,0,0,0,1,0,0,0,0,0,12
4815,Cornish Pasty,http://www.seriouseats.com/recipes/2012/01/cor...,4.0,1069.150952,100.0,499.039135,267.287738,25.000,10.440516,2.639034,...,0,0,0,1,0,0,0,0,0,12
8836,Spinach Baked Eggs Recipe,http://www.seriouseats.com/recipes/2011/07/spi...,4.0,583.560943,35.0,169.299428,145.890236,8.750,3.367449,1.173786,...,0,0,0,1,0,0,0,0,0,12
8835,Croissants,http://www.seriouseats.com/recipes/2011/08/how...,10.0,782.147968,840.0,201.172133,78.214797,84.000,1.057124,0.404750,...,0,0,0,2,0,0,0,0,0,11
8140,Chicken and Leek Pie Recipe,http://www.seriouseats.com/recipes/2013/03/chi...,4.0,1138.353567,120.0,967.744834,284.588392,30.000,33.669972,8.856714,...,0,0,0,1,0,0,0,0,0,11
1933,'Pizza' Bread Recipe,http://www.seriouseats.com/recipes/2012/06/piz...,16.0,890.351915,150.0,150.093894,55.646995,9.375,1.952873,0.439636,...,1,0,0,2,0,0,0,0,0,11


The recommended recipes look good.  I'm not sure why cupcakes showed up in the reommendation though...

In [122]:
#I feel like eating some Chinese food.  What French foods are in df_recs?
df_recs[df_recs['cuisine_type_Chinese'] == 1]['url'].head(10)

31     http://www.seriouseats.com/recipes/2012/01/hom...
58     http://www.seriouseats.com/recipes/2011/02/kor...
92     https://food52.com/recipes/62513-three-cup-chi...
132    https://food52.com/recipes/4736-chicken-brocco...
147    http://www.marthastewart.com/339865/sesame-noo...
183    http://www.epicurious.com/recipes/food/views/C...
184    http://www.epicurious.com/recipes/food/views/S...
198    http://www.epicurious.com/recipes/food/views/k...
202    http://www.epicurious.com/recipes/food/views/C...
207         https://food52.com/recipes/11594-tofu-burger
Name: url, dtype: object

In [129]:
#Let's take a look at the recipe at index 32:
url = df_recs.iloc[184]['url']
url

'http://www.epicurious.com/recipes/food/views/Spicy-Orange-Chicken-Stir-Fry-353398'

This recipe looks delicious!  I'd like to find similar recipes and to see what happens when I sort by health score (bias toward a healthier option).

In [130]:
recommender_health_bias(url)

Health score of original recipe: 184    12
Name: nutrition_score, dtype: int64
Top 10 results before adding the health bias: url
http://www.epicurious.com/recipes/food/views/Spicy-Orange-Chicken-Stir-Fry-353398    1.000000
https://www.allrecipes.com/recipe/16406/orange-chicken-stir-fry/                     0.769531
http://www.bigoven.com/recipe/Chicken-Stir-Fry/189299                                0.762695
http://www.bigoven.com/recipe/chicken-stir-fry/189299                                0.761230
https://www.allrecipes.com/recipe/118579/orange-chicken-stir-fry/                    0.749512
http://www.epicurious.com/recipes/food/views/Chicken-Lettuce-Cups-369491             0.737305
https://www.allrecipes.com/recipe/230250/orange-chicken-stir-fry/                    0.735352
http://www.food.com/recipe/orange-chicken-518534                                     0.713867
http://www.epicurious.com/recipes/food/views/easy-general-tsos-chicken               0.698242
http://www.myrecipes.com/

,title,url,yield,total_weight,total_time,calories_per_serv,total_weight_per_serv,total_time_per_serv,monounsat_fat_g_per_serv,polyunsat_fat_g_per_serv,...,ingredient_category_water,ingredient_category_wines,ingredient_category_yogurt,ingredient_category_condiments_and_sauces,ingredient_category_cured_meats,ingredient_category_frozen_poultry,ingredient_category_plant_based_protein,ingredient_category_liquors_and_cocktails,ingredient_category_sugar_and_syrups,nutrition_score
10805,Chicken Tofu Stir Fry,http://www.recipezaar.com/Chicken-Tofu-Stir-Fr...,8.0,3371.922818,0.0,351.386038,421.490352,0.000,6.967418,5.178315,...,0,0,0,0,0,0,1,0,0,21
7307,Orange Stir-Fry Chicken,http://www.goodhousekeeping.com/food-recipes/a...,4.0,1878.209037,0.0,498.449378,469.552259,0.000,7.183176,3.623544,...,0,0,0,1,0,0,1,0,1,20
11043,Chicken-Orange Stir-Fry,http://www.myrecipes.com/recipe/chicken-orange...,4.0,2033.348069,0.0,415.851439,508.337017,0.000,4.894613,2.459228,...,0,0,0,0,0,0,1,0,0,20
1520,Fried Rice with Pineapple and Tofu,http://www.myrecipes.com/recipe/fried-rice-wit...,7.0,2176.996644,0.0,586.543412,310.999521,0.000,4.273704,5.381938,...,0,0,0,1,0,0,1,0,0,19
338,Chicken And White Bean Chili,http://www.epicurious.com/recipes/food/views/C...,10.0,4032.928531,0.0,550.013105,403.292853,0.000,8.094677,2.169284,...,0,0,0,1,0,0,1,0,0,18
18952,Orange Chicken Stir-Fry,https://www.allrecipes.com/recipe/230250/orang...,4.0,2199.392370,40.0,581.296627,549.848092,10.000,9.681866,3.421431,...,0,0,0,1,0,0,1,0,0,18
3918,Oven-Roasted Root Vegetables,https://www.allrecipes.com/recipe/179055/oven-...,8.0,3450.369480,85.0,199.786025,431.296185,10.625,1.346525,0.485308,...,0,0,0,1,0,0,0,0,0,18
6566,Sweet-And-Sour Pork Stir-Fry,http://www.marthastewart.com/336770/sweet-and-...,4.0,1396.831992,0.0,280.810094,349.207998,0.000,6.546390,2.033573,...,0,0,0,1,0,0,1,0,0,17
10377,Orange Chicken Stir-Fry,https://www.allrecipes.com/recipe/118579/orang...,2.0,936.600000,20.0,625.885750,468.300000,10.000,13.333113,10.552842,...,0,0,0,1,0,0,1,0,0,17
8248,Orange Chicken,http://www.food.com/recipe/orange-chicken-439409,4.0,2086.220784,90.0,1290.034080,521.555196,22.500,48.215005,20.654098,...,0,0,0,1,0,0,0,0,0,16


The recommended recipes from the unbiased and biased recommender outputs both look great!  The chili recipe doesn't seem as relevant as other outputs, but most of the recipes look relevant.

In [141]:
#I feel like eating some Japanese food.  What Japanese foods are in df_recs?
df_recs[df_recs['cuisine_type_Japanese'] == 1]['url'].head(20)

200     http://www.kitchendaily.com/recipe/quick-chick...
236     http://www.cookstr.com/Seafood-Recipes/Marinat...
267     https://food52.com/recipes/57582-ginger-miso-d...
457       http://www.cookstr.com/recipes/chicken-yakitori
538     http://www.kitchendaily.com/recipe/yakitori-ch...
751     http://www.delish.com/cooking/recipe-ideas/rec...
752      http://www.myrecipes.com/recipe/chicken-yakitori
793     https://www.foodnetwork.com/recipes/tyler-flor...
867     http://www.myrecipes.com/recipe/chicken-yakito...
913     http://www.myrecipes.com/recipe/korean-beef-ri...
971     https://www.foodnetwork.com/recipes/alton-brow...
972     http://www.delish.com/cooking/recipe-ideas/rec...
1084    http://www.myrecipes.com/recipe/asian-turkey-l...
1132    http://www.grouprecipes.com/78000/sushi-rice.html
1190    https://www.allrecipes.com/recipe/116616/grill...
1192    http://www.tasteofhome.com/Recipes/Grilled-Ter...
1241    http://www.williams-sonoma.com/recipe/grilled-...
1424    http:/

In [148]:
#Let's take a look at the recipe at index 1590:
url = df_recs.iloc[1590]['url']
url

'http://allrecipes.com/Recipe/Grilled-Marinated-Salmon/Detail.aspx'

This recipe looks delicious!  I'd like to find similar recipes and to see what happens when I sort by health score (bias toward a healthier option).

In [149]:
recommender_health_bias(url)

Health score of original recipe: 1590    11
Name: nutrition_score, dtype: int64
Top 10 results before adding the health bias: url
http://allrecipes.com/Recipe/Grilled-Marinated-Salmon/Detail.aspx    1.000000
http://www.food.com/recipe/grilled-marinated-salmon-459315           0.980957
https://www.allrecipes.com/recipe/160929/maple-soy-glazed-salmon/    0.881836
https://www.allrecipes.com/recipe/122640/teriyaki-salmon/            0.871582
http://www.food.com/recipe/wasabi-salmon-304464                      0.870117
http://www.bigoven.com/recipe/Miso-Salmon/180044                     0.869629
http://www.food.com/recipe/teriyaki-salmon-446308                    0.865234
http://www.food.com/recipe/wasabi-salmon-431210                      0.850586
http://www.tasteofhome.com/Recipes/Teriyaki-Salmon                   0.841797
http://www.food.com/recipe/miso-salmon-233282                        0.820801
Name: http://allrecipes.com/Recipe/Grilled-Marinated-Salmon/Detail.aspx, dtype: float16
To

,title,url,yield,total_weight,total_time,calories_per_serv,total_weight_per_serv,total_time_per_serv,monounsat_fat_g_per_serv,polyunsat_fat_g_per_serv,trans_fat_g_per_serv,sugar_g_per_serv,calcium_pct_div100_per_serv,carbs_pct_div100_per_serv,cholesterol_pct_div100_per_serv,energy_pct_div100_per_serv,sat_fat_pct_div100_per_serv,fat_pct_div100_per_serv,iron_pct_div100_per_serv,fiber_pct_div100_per_serv,folate_pct_div100_per_serv,potassium_pct_div100_per_serv,magnesium_pct_div100_per_serv,sodium_pct_div100_per_serv,niacin_pct_div100_per_serv,phosphorus_pct_div100_per_serv,protein_pct_div100_per_serv,riboflavin_pct_div100_per_serv,thiamin_pct_div100_per_serv,vit_A_pct_div100_per_serv,vit_B6_pct_div100_per_serv,vit_B12_pct_div100_per_serv,vit_C_pct_div100_per_serv,vit_D_pct_div100_per_serv,vit_E_pct_div100_per_serv,vit_K_pct_div100_per_serv,source_BigOven,source_Cookstr,source_Delish,source_Epicurious,source_Food Network,source_Food52,source_Foodista,source_Good Housekeeping,source_Group Recipes,source_Kitchen Daily,source_Kraft Foods,source_Martha Stewart,source_My Recipes,source_Saveur,source_Serious Eats,source_Taste of Home,source_Williams-Sonoma,source_food.com,source_recipezaar.com,source_nan,diet_labels_Balanced,diet_labels_High-Fiber,diet_labels_High-Protein,diet_labels_Low-Carb,diet_labels_Low-Fat,diet_labels_Low-Sodium,health_labels_Alcohol-Cocktail,health_labels_Alcohol-Free,health_labels_Celery-Free,health_labels_Crustacean-Free,health_labels_Dairy-Free,health_labels_Egg-Free,health_labels_Fish-Free,health_labels_Gluten-Free,health_labels_Keto-Friendly,health_labels_Kidney-Friendly,health_labels_Kosher,health_labels_Low Potassium,health_labels_Low Sugar,health_labels_Lupine-Free,health_labels_Mollusk-Free,health_labels_Mustard-Free,health_labels_No oil added,health_labels_Paleo,health_labels_Peanut-Free,health_labels_Pescatarian,health_labels_Pork-Free,health_labels_Red-Meat-Free,health_labels_Sesame-Free,health_labels_Shellfish-Free,health_labels_Soy-Free,health_labels_Sugar-Conscious,health_labels_Tree-Nut-Free,health_labels_Vegan,health_labels_Vegetarian,health_labels_Wheat-Free,cautions_FODMAP,cautions_Sulfites,cuisine_type_.,cuisine_type_4 Points,cuisine_type_African,cuisine_type_Albanian,cuisine_type_Amerian,cuisine_type_American,"cuisine_type_American, Barbecue, Southern","cuisine_type_American, French","cuisine_type_American, Italian","cuisine_type_American, Southern",cuisine_type_American-South,cuisine_type_American|Asian,cuisine_type_American|North American|Cajun/Creole,cuisine_type_Amish,cuisine_type_Andhra,cuisine_type_Appetizer,cuisine_type_Arab,cuisine_type_Argentinian,cuisine_type_Armenian,cuisine_type_Asian,"cuisine_type_Asian, Chinese",cuisine_type_Aussie,cuisine_type_Australian,cuisine_type_Australian/New Zealand,cuisine_type_Austrian,cuisine_type_Baking,cuisine_type_Balochi,cuisine_type_Bangladeshi,cuisine_type_Belarusian,cuisine_type_Belgian,cuisine_type_Belgium,cuisine_type_Brazilian,cuisine_type_Breakfast,cuisine_type_British,cuisine_type_BritishEurope,cuisine_type_Bulgarian,cuisine_type_Cajun,cuisine_type_Cajun/Creole|Southern,cuisine_type_Canadian,cuisine_type_Caribbean,cuisine_type_Central American/Caribbean,cuisine_type_Cheap,cuisine_type_Chechen,cuisine_type_Chinese,cuisine_type_Colombian,cuisine_type_Cookie,cuisine_type_Costa Rican,cuisine_type_Creole,cuisine_type_Cuban,cuisine_type_Cuban|Central American/Caribbean,cuisine_type_Curry,cuisine_type_Danish,cuisine_type_Dessert,cuisine_type_Dominican,cuisine_type_Dutch,cuisine_type_Earth,cuisine_type_Eastern Europe,cuisine_type_Eastern European,cuisine_type_Ecuadorian,cuisine_type_Egypt,cuisine_type_English,cuisine_type_Estern,cuisine_type_Europe,cuisine_type_European,"cuisine_type_European, French, Paris","cuisine_type_European, Italian","cuisine_type_European, Scottish",cuisine_type_Filipino,cuisine_type_France,cuisine_type_French,cuisine_type_GAPS,cuisine_type_German,cuisine_type_Goan,cuisine_type_Gourment,cuisine_type_Greek,cuisine_type_Hawa

The unbiased and biased recommended recipes look great!  I'm amazed that there are so many salmon recipes output here.

In [173]:
#I feel like eating some American food.  What American foods are in df_recs?
df_recs[df_recs['cuisine_type_American'] == 1]['url'].head(30)

2      http://www.saveur.com/article/Recipes/Negroni-...
8      http://www.saveur.com/article/Recipes/Manhatta...
14     http://www.marthastewart.com/314029/blood-oran...
21     http://www.marthastewart.com/316100/homemade-h...
22     http://www.seriouseats.com/recipes/2012/01/duc...
25     http://www.seriouseats.com/recipes/2010/10/the...
29     http://www.seriouseats.com/recipes/2011/03/hom...
42     http://www.marthastewart.com/874841/mushroom-b...
46     http://www.seriouseats.com/recipes/2010/10/new...
50     http://www.epicurious.com/recipes/food/views/G...
51     http://www.marthastewart.com/314217/boiled-shrimp
52     http://www.seriouseats.com/recipes/2011/04/bla...
56     http://www.marthastewart.com/344408/beef-tende...
57     http://www.epicurious.com/recipes/food/views/T...
68     http://www.marthastewart.com/336066/pork-loin-...
76     http://www.marthastewart.com/343883/balsamic-c...
83     https://food52.com/recipes/7060-mushroom-meatloaf
89     http://www.marthastewart

In [174]:
#Let's take a look at the recipe at index 106:
url = df_recs.iloc[106]['url']
url

'http://www.epicurious.com/recipes/food/views/Buffalo-Chicken-Wings-101925'

This recipe looks delicious!  I'd like to find similar recipes and to see what happens when I sort by health score (bias toward a healthier option).

In [175]:
recommender_health_bias(url)

Health score of original recipe: 106    7
Name: nutrition_score, dtype: int64
Top 10 results before adding the health bias: url
http://www.epicurious.com/recipes/food/views/Buffalo-Chicken-Wings-101925            1.000000
http://www.epicurious.com/recipes/food/views/Green-Goddess-Dressing-106301           0.699219
http://www.epicurious.com/recipes/food/views/Tandoori-Chicken-240025                 0.611816
http://www.epicurious.com/recipes/food/views/Chicken-and-Dumplings-240818            0.582520
http://www.epicurious.com/recipes/food/views/Hot-Wings-368590                        0.578125
http://www.epicurious.com/recipes/food/views/pork-chops-with-mustard-sauce-231998    0.548828
http://www.epicurious.com/recipes/food/views/Chicken-Tikka-109308                    0.530273
http://www.food.com/recipe/hot-artichoke-dip-131279                                  0.522461
http://www.epicurious.com/recipes/food/views/cloud-bread                             0.519043
http://www.epicurious.com/

,title,url,yield,total_weight,total_time,calories_per_serv,total_weight_per_serv,total_time_per_serv,monounsat_fat_g_per_serv,polyunsat_fat_g_per_serv,trans_fat_g_per_serv,sugar_g_per_serv,calcium_pct_div100_per_serv,carbs_pct_div100_per_serv,cholesterol_pct_div100_per_serv,energy_pct_div100_per_serv,sat_fat_pct_div100_per_serv,fat_pct_div100_per_serv,iron_pct_div100_per_serv,fiber_pct_div100_per_serv,folate_pct_div100_per_serv,potassium_pct_div100_per_serv,magnesium_pct_div100_per_serv,sodium_pct_div100_per_serv,niacin_pct_div100_per_serv,phosphorus_pct_div100_per_serv,protein_pct_div100_per_serv,riboflavin_pct_div100_per_serv,thiamin_pct_div100_per_serv,vit_A_pct_div100_per_serv,vit_B6_pct_div100_per_serv,vit_B12_pct_div100_per_serv,vit_C_pct_div100_per_serv,vit_D_pct_div100_per_serv,vit_E_pct_div100_per_serv,vit_K_pct_div100_per_serv,source_BigOven,source_Cookstr,source_Delish,source_Epicurious,source_Food Network,source_Food52,source_Foodista,source_Good Housekeeping,source_Group Recipes,source_Kitchen Daily,source_Kraft Foods,source_Martha Stewart,source_My Recipes,source_Saveur,source_Serious Eats,source_Taste of Home,source_Williams-Sonoma,source_food.com,source_recipezaar.com,source_nan,diet_labels_Balanced,diet_labels_High-Fiber,diet_labels_High-Protein,diet_labels_Low-Carb,diet_labels_Low-Fat,diet_labels_Low-Sodium,health_labels_Alcohol-Cocktail,health_labels_Alcohol-Free,health_labels_Celery-Free,health_labels_Crustacean-Free,health_labels_Dairy-Free,health_labels_Egg-Free,health_labels_Fish-Free,health_labels_Gluten-Free,health_labels_Keto-Friendly,health_labels_Kidney-Friendly,health_labels_Kosher,health_labels_Low Potassium,health_labels_Low Sugar,health_labels_Lupine-Free,health_labels_Mollusk-Free,health_labels_Mustard-Free,health_labels_No oil added,health_labels_Paleo,health_labels_Peanut-Free,health_labels_Pescatarian,health_labels_Pork-Free,health_labels_Red-Meat-Free,health_labels_Sesame-Free,health_labels_Shellfish-Free,health_labels_Soy-Free,health_labels_Sugar-Conscious,health_labels_Tree-Nut-Free,health_labels_Vegan,health_labels_Vegetarian,health_labels_Wheat-Free,cautions_FODMAP,cautions_Sulfites,cuisine_type_.,cuisine_type_4 Points,cuisine_type_African,cuisine_type_Albanian,cuisine_type_Amerian,cuisine_type_American,"cuisine_type_American, Barbecue, Southern","cuisine_type_American, French","cuisine_type_American, Italian","cuisine_type_American, Southern",cuisine_type_American-South,cuisine_type_American|Asian,cuisine_type_American|North American|Cajun/Creole,cuisine_type_Amish,cuisine_type_Andhra,cuisine_type_Appetizer,cuisine_type_Arab,cuisine_type_Argentinian,cuisine_type_Armenian,cuisine_type_Asian,"cuisine_type_Asian, Chinese",cuisine_type_Aussie,cuisine_type_Australian,cuisine_type_Australian/New Zealand,cuisine_type_Austrian,cuisine_type_Baking,cuisine_type_Balochi,cuisine_type_Bangladeshi,cuisine_type_Belarusian,cuisine_type_Belgian,cuisine_type_Belgium,cuisine_type_Brazilian,cuisine_type_Breakfast,cuisine_type_British,cuisine_type_BritishEurope,cuisine_type_Bulgarian,cuisine_type_Cajun,cuisine_type_Cajun/Creole|Southern,cuisine_type_Canadian,cuisine_type_Caribbean,cuisine_type_Central American/Caribbean,cuisine_type_Cheap,cuisine_type_Chechen,cuisine_type_Chinese,cuisine_type_Colombian,cuisine_type_Cookie,cuisine_type_Costa Rican,cuisine_type_Creole,cuisine_type_Cuban,cuisine_type_Cuban|Central American/Caribbean,cuisine_type_Curry,cuisine_type_Danish,cuisine_type_Dessert,cuisine_type_Dominican,cuisine_type_Dutch,cuisine_type_Earth,cuisine_type_Eastern Europe,cuisine_type_Eastern European,cuisine_type_Ecuadorian,cuisine_type_Egypt,cuisine_type_English,cuisine_type_Estern,cuisine_type_Europe,cuisine_type_European,"cuisine_type_European, French, Paris","cuisine_type_European, Italian","cuisine_type_European, Scottish",cuisine_type_Filipino,cuisine_type_France,cuisine_type_French,cuisine_type_GAPS,cuisine_type_German,cuisine_type_Goan,cuisine_type_Gourment,cuisine_type_Greek,cuisine_type_Hawa

The unbiased and biased recommendations look pretty good.  I was hoping to see more healthy chicken wings recipes output.  Maybe this means chicken wings are not as healthy as I hoped...

The recommender seems to be doing well.  I think it would be helpful to analyze how accurate the nutrition score is to make sure the recommender is accurately recommending recipes that are indeed healthier than the initial unbiased recommended recipes.